# Dynamics via acceleration maps

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from analysis import analysis

from importlib import reload

L_box = 50
mu_factor = 6  # conversion to microns
min_factor = 8  # conversion to minutes


#### Load grid by grid
- Load `fulltake_gid*.pkl`, which is all of the runs for a config

In [ ]:
from glob import glob

files = glob("../_server/fulltake_gid*.pkl")
data = []

for file in files:
    df = pd.read_pickle(file)

    df.x *= mu_factor
    df.y *= mu_factor
    data.append(df)

print(f"Loaded {len(data)} configurations.")
[
    print(
        f"\t - Grid {df.gid.iloc[0]}: {len(df)} total data points | {len(df.rid.unique())} runs | {len(df[df.rid==0])} data points / run"
    )
    for df in data
]
display(data[0])


#### Calculate $\{v, a\}$ and bin $\{x, v, a\}$
- `global_x_v_a`: `list`, with each element being a `pd.DataFrame` for an entire grid parameter. The data frame contains the position and speed for all the runs. 

In [ ]:
global_x_v_a = []

for df_gid in data:

    # x, v, a values for this entire config
    grid_x_v_a = []

    # compute speed and acc for each run
    for rid, df_rid in df_gid.groupby("rid"):
        x_v_a = analysis.calc_v_a_from_position(df_rid.x, df_rid["time[hr]"])
        x_v_a[df_gid.iloc[0][5:].index] = data[0].iloc[0][5:]
        grid_x_v_a.append(x_v_a)

    grid_x_v_a = pd.concat(grid_x_v_a)

    # with a 112mu mp, this gives bins of dim 3.5mu x 3.5mu
    nbins = 32
    analysis.get_bin_indices(grid_x_v_a, nbins)

    global_x_v_a.append(grid_x_v_a)

# global_x_v_a = pd.concat(global_x_v_a)
global_x_v_a[0]


#### Acceleration maps and flow lines

- Consider each grid's behavior aggregated across all its runs.

In [ ]:
def _make_title(df):
    tbl = {
        "gamma": r"$\gamma$",
        "R_eq": r"$R_{eq}$",
        "mag_std": r"$\sigma_{MVG}$",
        "add_rate": r"$\tau_{MVG}$",
        "gid": "ID",
    }
    d = dict(df.iloc[0][3:8])
    title = ""
    for key, val in d.items():
        title += tbl[key] + " = " + f"{val}" + "\n"
    return title


In [ ]:
dt = 0.0015 * 250 * 8 / 60  # hr

gids = [0, 1]
for gid in gids:
    F, sigma = analysis.calc_F_sigma(global_x_v_a[gid], dt, nbins, min_pts=2)

    bounds = global_x_v_a[gid].agg(["min", "max"])
    xmin, xmax = bounds["x"]
    vmin, vmax = bounds["v"]

    plot_title = _make_title(global_x_v_a[gid])
    bounds = (xmin, xmax, vmin, vmax, nbins)
    title = {"title": plot_title, "size": 20}

    analysis.imshow_F_sigma([F, sigma], bounds, title, interp="bilinear")


#### Position dist

In [ ]:
reload(analysis)

plt.rc("axes", titlesize=8)
analysis.position_dist(data, "two_state", mu_factor)


#### Hopping times

In [ ]:
fig, axs = plt.subplots(3, 3, figsize=(8, 6), dpi=200)
plt.rc("axes", titlesize=12)

x, y = np.meshgrid([0, 1, 2], [0, 1, 2])

# in microns
d = 73 / 2
xL = (L_box / 2) * 6 - d
xR = (L_box / 2) * 6 + d
buffer = 38 / 2

for i, j, df in zip(x.flatten(), y.flatten(), data):
    beta = df.beta.iloc[0]
    gamma = df.gamma.iloc[0]
    D = df.D.iloc[0]
    axs[i, j].set_title(rf"$\beta$ = {beta}, $\gamma$ = {gamma}, $D$ = {D}", y=1.1)

    hop_times = []
    for rid, d in df.groupby("rid"):
        for t in analysis.get_hopping_times(d, xL, xR, buffer):
            hop_times.append(t)

    if len(hop_times) == 0:
        continue

    axs[i, j].hist(hop_times, color="red", alpha=0.5, density=True)
    axs[i, j].set_xlabel("Hopping time (hr)", fontsize=14)

plt.tight_layout()
plt.subplots_adjust(hspace=2, wspace=2)
plt.show()
